In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [2]:
gnn_train_data = pd.read_parquet('../data/curated/ML_data/gnn_train_data.parquet')
gnn_val_data = pd.read_parquet('../data/curated/ML_data/gnn_val_data.parquet')
gnn_test_data = pd.read_parquet('../data/curated/ML_data/gnn_test_data.parquet')

SA2_gnn_data = pd.read_parquet('../data/curated/ML_data/SA2_gnn_data.parquet')
station_inference_gnn_data = pd.read_parquet('../data/curated/ML_data/station_inference_gnn_data.parquet')

station_inference_gnn_data = station_inference_gnn_data.rename({'Station_Na': 'Station Name'}, axis=1)
inference_data = pd.concat([SA2_gnn_data, station_inference_gnn_data], axis=0)
inference_data = inference_data.rename({'Station Name': 'Station_Name'}, axis=1)

In [3]:
# stations_list = [x for x in gnn_train_data['Station_Name'].unique()]
# stations_index = {stations_list[i]:i for i in range(len(stations_list))}
# reverse_stations_index = {v: k for (k, v) in stations_index.items()}

# open npy
station_weights_matrix = np.load('../data/curated/ML_data/station_weights_matrix.npy')
SA2_weights_matrix = np.load('../data/curated/ML_data/station_weights_withSA2_matrix.npy')

with open('../data/curated/ML_features/station_weights_withSA2.json', 'r') as f:
    station_weights_withSA2 = json.load(f)

with open('../data/curated/ML_features/station_weights.json', 'r') as f:
    station_weights = json.load(f)

In [4]:
geospatial_features = ['log_Total_Demand']
non_geospatial_features = ['Weekday', 'PublicHoliday', 'mean_rainfall_value', 'has_school',
       'has_sport_facility', 'has_shopping_centre', 'has_hospital',
       'total_population', ' med_rent_weekly_c2021',
       ' med_mortg_rep_mon_c2021', ' med_person_inc_we_c2021',
       ' med_famly_inc_we_c2021']
label_columns = ['log_Total_Demand']

In [5]:
def DataFactory(raw_dataset, geospatial_features, non_geospatial_features, label_columns, stations_index, inference = False):

    """ Data Factory of GNN """
    
    geospatial_x_batches = []
    non_geospatial_x_batches = []
    y_batches = []
    masks = []

    if inference:
        groupby_column = 'Weekday'
    else:
        groupby_column = 'Business_Date'

    for day, daily_df in tqdm(raw_dataset.groupby([groupby_column])):

        geospatial_x = np.zeros([len(stations_index), len(geospatial_features)])
        y = np.zeros([len(stations_index), len(label_columns)])
        mask = np.zeros([len(stations_index), 1])
        non_geospatial_x = np.zeros([len(stations_index), len(non_geospatial_features)])

        daily_df.set_index('Station_Name', inplace=True)

        for station in daily_df.index:

            geospatial_x[stations_index[station]] = daily_df.loc[station][geospatial_features] # todo inference. 
            if not inference:
                y[stations_index[station]] = daily_df.loc[station][label_columns]
            mask[stations_index[station]] = 1
            non_geospatial_x[stations_index[station]] = daily_df.loc[station][non_geospatial_features]
                
        geospatial_x_batches.append(geospatial_x)
        y_batches.append(y)
        masks.append(mask.flatten())

        non_geospatial_x_batches.append(non_geospatial_x)

        
    return geospatial_x_batches, non_geospatial_x_batches, y_batches, masks

In [6]:
train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks = DataFactory(gnn_train_data, geospatial_features, non_geospatial_features, label_columns, station_weights)
val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks = DataFactory(gnn_val_data, geospatial_features, non_geospatial_features, label_columns, station_weights)
test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks = DataFactory(gnn_test_data, geospatial_features, non_geospatial_features, label_columns, station_weights)

# inference_geospatial_X_batches, inference_non_geospatial_X_batches, inference_y_batches, inference_masks = DataFactory(inference_data, geospatial_features, non_geospatial_features, label_columns, station_weights_withSA2, inference = True)

100%|██████████| 82/82 [00:14<00:00,  5.68it/s]


In [7]:
import sys
import os

py_file_location = '../'
home_directory = '../'

sys.path.append(os.path.abspath(py_file_location))
from model.model_class.environment import *

In [8]:
from model.model_class import GNN

In [9]:
SEED = 42

In [13]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0.1
    n_gnn_layers = 1
    activation = nn.ReLU()
    res_learning = False
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 10
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 12
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'GNN'

model = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

  0%|          | 0/382 [00:00<?, ?it/s]

100%|██████████| 382/382 [00:13<00:00, 28.85it/s]


 Epoch 1 Train | Loss:  0.0368 | R2:  0.9598| MSE:  0.0369 | RMSE:  0.1921 | MAE:  0.1140 


100%|██████████| 82/82 [00:00<00:00, 89.37it/s]


Epoch 1 Val | Loss:  0.0066 | R2:  0.9934| MSE:  0.0066 | RMSE:  0.0811 | MAE:  0.0679 


...

Epoch 14 Val | Loss:  0.0023 | R2:  0.9977| MSE:  0.0023 | RMSE:  0.0475 | MAE:  0.0325 


100%|██████████| 82/82 [00:00<00:00, 86.65it/s]


Epoch 7 Val | Loss:  0.0021 | R2:  0.9979| MSE:  0.0021 | RMSE:  0.0460 | MAE:  0.0343 


100%|██████████| 82/82 [00:00<00:00, 104.26it/s]


Epoch 7 Val | Loss:  0.0019 | R2:  0.9979| MSE:  0.0019 | RMSE:  0.0438 | MAE:  0.0332 
